In [1]:
import math
import numpy as np
import pandas as pd
import timeit

In [2]:
citiesnp = np.genfromtxt('cities.csv', delimiter=',', skip_header = 1) # load as numpy array

In [3]:
def not_prime(n):
    if n == 2:
        return False
    if n % 2 == 0 or n <= 1:
        return True

    sqr = int(math.sqrt(n)) + 1

    for divisor in range(3, sqr, 2):
        if n % divisor == 0:
            return True
    return False

In [4]:
np_not_prime = np.vectorize(not_prime)

In [5]:
def np_total_length(c, r):
    r = np.concatenate(([0], r))
    c = c[r, :]
    cs = np.roll(c, -1, axis =0)
    d = np.sqrt((c[:,1] - cs[:,1])**2 + (c[:,2] - cs[:,2])**2)
    m = np.concatenate((np.arange(1, len(c)+1)[np.newaxis,:],
                        cs[:, 0][np.newaxis,:],
                        d[np.newaxis,:]))
    p = m[:, np.arange(9, len(c), 10)]
    sel = p[0,][np_not_prime(p[1,])].astype(int)
    m[:, sel-1] *= 1.1
    
    return np.sum(m[2])

In [6]:
def np_total_length2(c, r):
    r = np.concatenate(([0], r))
    c = c[r, :]
    cs = np.roll(c, -1, axis =0)
    d = np.sqrt(np.power(c[:,1] - cs[:,1], 2) + np.power(c[:,2] - cs[:,2], 2))
    m = np.concatenate((np.arange(1, len(c)+1)[np.newaxis,:],
                        cs[:, 0][np.newaxis,:],
                        d[np.newaxis,:]))    
    p = m[:, np.arange(9, len(c), 10)]
    sel = p[0,][np_not_prime(p[1,])].astype(int)
    m[:, sel-1] *= 1.1
    
    return np.sum(m[2])

Best performing old funcion:

In [7]:
def total_length_mod(coors, route):
    route = np.concatenate(([0], route, [0]))   
    df = coors.copy()
    df = df.loc[route].reset_index()
    df = df.rename(columns = {'index' : 'CityId'})
    df['dist'] = np.sqrt((df.X - df.X.shift())**2 + (df.Y - df.Y.shift())**2)
    df = df.drop(0)
    idx = (df.index % 10 == 0)
    idx = df.loc[idx].CityId.apply(not_prime)
    idx = idx.index[idx.values]
    df.loc[idx, 'dist'] = df.loc[idx, 'dist'] + df.loc[idx, 'dist'] / 10
    # print(df)
    return np.sum(df['dist'])

In [8]:
def total_length_mod2(coors, route):
    route = np.concatenate(([0], route, [0]))   
    df = coors.copy()
    df = df.loc[route].reset_index()
    df = df.rename(columns = {'index' : 'CityId'})
    df['dist'] = np.sqrt(
        np.power(df.X - df.X.shift(), 2) + np.power(df.Y - df.Y.shift(), 2))
    df = df.drop(0)
    idx = (df.index % 10 == 0)
    idx = df.loc[idx].CityId.apply(not_prime)
    idx = idx.index[idx.values]
    df.loc[idx, 'dist'] *= 1.1
    # print(df)
    return np.sum(df['dist'])

# Performance

In [9]:
cities = pd.read_csv('cities.csv')
coors = cities[['X', 'Y']]

In [10]:
np.random.seed(3)
idx = np.random.permutation(range(1, len(citiesnp)))

In [11]:
np_total_length(citiesnp, idx)

447277748.2547363

In [12]:
np_total_length2(citiesnp, idx)

447277748.2547363

In [13]:
total_length_mod(coors, idx)

447277748.2547363

In [14]:
total_length_mod2(coors, idx)

447277748.2547363

In [15]:
timeit.timeit('np_total_length(citiesnp, idx)', number=200, globals=globals())

13.9412906773715

In [16]:
timeit.timeit('np_total_length2(citiesnp, idx)', number=200, globals=globals())

18.983418715274084

In [17]:
timeit.timeit('total_length_mod(coors, idx)', number=200, globals=globals())

27.78198139587804

In [18]:
timeit.timeit('total_length_mod2(coors, idx)', number=200, globals=globals())

27.89765581353285